In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
import pandas as pd
import glob
import numpy as np
import calendar
from datetime import datetime
import os

# Carga de datos

In [7]:
df = pd.read_csv('gps_filtered.csv', sep=',', header=0)
#df = pd.read_csv('drive/My Drive/ProyectoCienciaDeDatos/datos_gps_curso_ciencia_de_datos/gps_filtered.csv', sep=',', header=0)

In [8]:
df.head()

,Unnamed: 0,DistanciaInicio,DistanciaRuta,GPS_time,Ind1,Ind2,Ind3,Latitud,Longitud,Patente,Servicio,Unnamed: 0.1,Unnamed: 0.1.1
0,0,0,147,2019-08-01 00:00:28,0,5,0,-33.543559,-70.789180,BDXR-15,T506 00I,0,12057.0
1,1,0,140,2019-08-01 00:00:15,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,1,12058.0
2,2,0,140,2019-08-01 00:00:45,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,2,12059.0
3,3,0,140,2019-08-01 00:01:15,0,5,0,-33.543655,-70.788752,BFKB-85,T506 00I,3,12060.0
4,4,0,139,2019-08-01 00:01:45,5,5,0,-33.543637,-70.788783,BFKB-85,T506 00I,4,12061.0


# Patentes

Filtra patentes según si cumplen formato XXXX-00 o XX-0000 

In [ ]:
def patente(df):
    """
    df: dataframe
    
    return: dataframe filtrado con formato de patente correcto
    (XXXX-00 o XX-0000)
    """
    df = df[df['Patente'].map(lambda x: (x[:4].isalpha() and x[5:].isnumeric()) or (x[:2].isalpha() and x[4:].isnumeric()))]

# Servicios  

formatoServicio revisa formato según orden de caracteres (letra, número o espacio)

In [ ]:
#--machine <patente> <servicio> <distancia_en_ruta> <tiempo_medición>
def formatoServicio(servicio):
    """
    servicio: string que indica el servicio
    
    return: True si el servicio cumple el formato, False si no
    """
    
    #ej: B09 00R
    if len(servicio) == 7:
        if servicio[0].isalpha() and servicio[1:3].isnumeric() and servicio[3].isspace() \
        and servicio[4:6].isnumeric() and servicio[6].isalpha():
            return True

        else: 
            return False

    #ej: T201 00I, B04V 00I
    if len(servicio) == 8:
        if servicio[0].isalpha() and servicio[1:4].isnumeric() and servicio[4].isspace() \
        and servicio[5:7].isnumeric() and servicio[7].isalpha():
            return True

        if servicio[0].isalpha() and servicio[1:3].isnumeric() and servicio[3].isalpha() \
        and servicio[4].isspace() and servicio[5:7].isnumeric() and servicio[7].isalpha():
            return True

        else: 
            return False

    #ej: B02 C2 00I
    if len(servicio) == 10:
        if servicio[0].isalpha() and servicio[1:3].isnumeric() and servicio[3].isspace() \
        and servicio[4].isalpha() and servicio[5].isnumeric() and servicio[6].isspace() \
        and servicio[7:9].isnumeric() and servicio[9].isalpha():
            return True
        else: 
            return False

    #ej: T201 E0 00I
    if len(servicio) == 11:
        if servicio[0].isalpha() and servicio[1:4].isnumeric() and servicio[4].isspace() \
        and servicio[5].isalpha() and servicio[6].isnumeric() and servicio[7].isspace() \
        and servicio[8:10].isnumeric() and servicio[10].isalpha():
            return True
        else:
            return False

    else:
        return False

esServicio revisa si servicio ingresado está dentro de la lista de servicios únicos

In [ ]:
def esServicio(servicio):
    """
    servicio: string que indica el servicio
    
    return: True si el servicio está en la lista, False si no
    """
    #servs = np.load('servicios_unicos.npy')
    servs = np.load('drive/My Drive/ProyectoCienciaDeDatos/datos_gps_curso_ciencia_de_datos/servicios_unicos.npy')
    if servicio in servs:
        return True
    else:
        return False

In [ ]:
def servicio(df, file):
    """
    df: dataframe
    
    return: dataframe servicios que cumplen
    formatoServicio y esServicio
    """
    #df = df[df['Servicio'].map(lambda x: (formatoServicio(x) == True) and (esServicio(x) == True))]
    df = df[df['Servicio'].map(lambda x: (x in file == True))]

#servs = np.load('drive/My Drive/ProyectoCienciaDeDatos/datos_gps_curso_ciencia_de_datos/servicios_unicos.npy')
#servicio(df, servs)
#len(df)

# Longitud y Latitud

In [ ]:
def esCoord(lon, lat):
    """
    lon: int, longitud
    lat: int, latitud
    
    return: True si el par está dentro del rango de 
    coordenadas de Santiago, False si no
    """
    
    if lon < -69 and lon > -71 and \
       lat < -32 and lat > -34:
        return True
    else:
        return False

In [ ]:
def coords(df):
    """
    df: dataframe
    
    return: dataframe coordenadas que están
    dentro de Santiago
    """
    df = df[df['Latitud'].map(lambda x: (x < -32) and (x > -34))]
    df = df[df['Longitud'].map(lambda x: (x < -69) and (x > -71))]

#coords(df)
#len(df)

# DistanciaInicio y DistanciaRuta

In [ ]:
def distancia(df):
    """
    df: dataframe
    
    return: dataframe con distanciainicio y distanciaruta 
    sin outliers (-1)
    """
    df = df[df['DistanciaInicio']>=0]
    df = df[df['DistanciaRuta']>=0]

# Fechas

Se filtran fechas que no correspondan al mes de Agosto del 2019

In [ ]:
def fechas(df):
    """
    df: dataframe
    
    return: dataframe que sólo conserva fechas del mes de Agosto 2019
    """
    df = df[df['GPS_time'].map(lambda x: (x[5:7] == '08') and (x[:4] == '2019'))]

In [ ]:
def gpsDia(GPS): 
    """
    GPS: dataframe con columna de fecha
    
    return: dataframe con una nueva columna que indica el 
    día de la semana de cada fecha
    """
    
    fecha = GPS['GPS_time']
    dias = []
    for i in fecha:
        d = calendar.day_name[i.weekday()]
        dias.append(d)
    GPS['dias']=dias

In [ ]:
def habilday(GPS):
    """
    GPS: dataframe
    
    return: columna que contiene un 1 si es dia habil y 0 si no 
    """
    dia_semana=['Monday','Tuesday','Wednesday','Thursday','Friday']
    dias=GPS['dias']
    dia_habil=[]
    for i in dias:
        if i in dia_semana:
            dia_habil.append(1)
        else:
            dia_habil.append(0)
    GPS['dia_habil']=dia_habil

# Eliminar columnas

In [ ]:
def delete_cols(df):
    """
    df: dataframe
    
    return: elimina columnas que no se utilizarán
    """
    df = df.drop(columns=['Unnamed:0','Ind1','Ind2','Ind3','Patente', 'GPS_time', 'dias'])

# Función de limpieza

In [ ]:
def limpieza(df, target = False):
    """
    df: dataframe que se quiere limpiar
    target: booleano que indica True si ya se creó la columna a predecir, False si no
    """
    if target:
        return delete_cols(df)
    else:
        df = patente(df)
        df = servicio(df)
        df = coords(df)
        df = distancia(df)
        df = gpsDia(df)
        return habilday(df)
